In [1]:
import tensorflow as tf
import re, json, pickle
from tqdm import tqdm
import numpy as np
import tensorflow_hub as hub
from keras.utils.vis_utils import plot_model
from keras.layers import merge
import pathlib
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pre_path = "/content/drive/MyDrive/Special_Topic/Dataset/MSCOCO_2017/"
# image_path = "/content/drive/MyDrive/University/Special_Topic/Dataste/MSCOCO/MSCOCO_2017/val2017"
image_path =  pre_path + "val2017/"
anno_path = pre_path + '2017_annotations/captions_train2017.json'
glove_path = pre_path + 'GloVe.6B/glove.6B.300d.txt'


In [4]:
results_path = '/content/drive/MyDrive/Special_Topic/Top_Down_Bttom_Up/Code/Results/Top-down_results_1.plk'
with open(results_path, 'rb') as f:
  results = pickle.loads(f.read())
# pickle.dump(results, outfile)

In [5]:
results

[{'caption': 'A cat sitting on a laptop computer on a desk.',
  'image_id': 504517},
 {'caption': 'A pair of scissors sitting on a wooden surface.',
  'image_id': 507068},
 {'caption': 'A cat sitting on a suitcase with a suitcase.',
  'image_id': 501377},
 {'caption': 'A group of elephants standing next to each other.',
  'image_id': 500214},
 {'caption': 'A table with a plate of food and a cup of coffee.',
  'image_id': 507939},
 {'caption': 'A group of people riding motorcycles on a street.',
  'image_id': 500032},
 {'caption': 'A man is flying a kite on a beach.', 'image_id': 507501},
 {'caption': 'A young boy is holding a frisbee in a field.',
  'image_id': 509712},
 {'caption': 'A living room with a couch and a table.', 'image_id': 503472},
 {'caption': 'A kitchen with a stove and a microwave.', 'image_id': 508481},
 {'caption': 'A man is playing frisbee in a field.', 'image_id': 505400},
 {'caption': 'A bathroom with a toilet and a sink.', 'image_id': 503200},
 {'caption': 'A man

In [6]:
image_list = list()
error = list()
for i  in range(len(results)):
  if results[i]['image_id'] in image_list:
    error.append(i)
    continue
  image_list.append(results[i]['image_id'])
error.reverse()
for i in error:
  del results[i]

In [7]:
generated_caption_path = "generated_caption_val2017.json"
with open(generated_caption_path, 'w') as jsonfile:
  json.dump(results, jsonfile)
  # json.dump(results[225:], jsonfile)

In [8]:
!git clone https://github.com/davidfsemedo/coco-caption


Cloning into 'coco-caption'...
remote: Enumerating objects: 737, done.
remote: Total 737 (delta 0), reused 0 (delta 0), pack-reused 737
Receiving objects: 100% (737/737), 106.72 MiB | 11.40 MiB/s, done.
Resolving deltas: 100% (400/400), done.


In [9]:
import os, sys
sys.path.insert(0,"/content/coco-caption")

In [10]:
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.3f')

In [11]:
#  workaround provided by https://github.com/tylin/coco-caption/issues/26#issuecomment-439825144
test_anno_path = pre_path + '2017_annotations/captions_val2017.json'
train_anno_path = pre_path + '2017_annotations/captions_train2017.json'
modified_anno_path = "/content/modified_captions.json"

with open(train_anno_path, 'r') as f:
    data = json.load(f)
    data['type'] = 'captions'
with open(modified_anno_path, 'w') as f:
    json.dump(data, f)

In [12]:
coco = COCO(modified_anno_path)
cocoRes = coco.loadRes(generated_caption_path)

loading annotations into memory...
0:00:01.360833
creating index...
index created!
Loading and preparing results...     
DONE (t=0.10s)
creating index...
index created!


In [13]:
results[69]

{'caption': 'A large building with a large building with a clock.',
 'image_id': 505933}

In [14]:
# create cocoEval object by taking coco and cocoRes
cocoEval = COCOEvalCap(coco, cocoRes)

# evaluate on a subset of images by setting
# cocoEval.params['image_id'] = cocoRes.getImgIds()
# please remove this line when evaluating the full validation set
cocoEval.params['image_id'] = cocoRes.getImgIds()

# evaluate results
cocoEval.evaluate()

tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 37884, 'reflen': 38028, 'guess': [37884, 33785, 29686, 25590], 'correct': [26346, 12972, 5387, 2147]}
ratio: 0.9962133165036028
Bleu_1: 0.693
Bleu_2: 0.515
Bleu_3: 0.363
Bleu_4: 0.252
computing METEOR score...
METEOR: 0.223
computing Rouge score...
ROUGE_L: 0.501
computing CIDEr score...
CIDEr: 0.775


In [15]:
# print output evaluation scores
for metric, score in cocoEval.eval.items():
    print('%s: %.3f'%(metric, score))

Bleu_1: 0.693
Bleu_2: 0.515
Bleu_3: 0.363
Bleu_4: 0.252
METEOR: 0.223
ROUGE_L: 0.501
CIDEr: 0.775
